In [13]:
# ==============================
# 1️⃣ Install dependencies
# ==============================
!pip install yolov5 easyocr opencv-python matplotlib --quiet

# ==============================
# 2️⃣ Imports
# ==============================
import torch
import cv2
import easyocr
import matplotlib.pyplot as plt
from google.colab import files

# YOLOv5 imports
import yolov5
import torch.nn as nn
from yolov5 import models

# ==============================
# 3️⃣ Dynamically register ALL YOLOv5 + torch.nn classes
# ==============================
import inspect

def add_all_classes_from_module(module):
    for name, obj in inspect.getmembers(module):
        if inspect.isclass(obj):
            torch.serialization.add_safe_globals([obj])

# Register YOLOv5 model classes
add_all_classes_from_module(models.yolo)
add_all_classes_from_module(models.common)

# Register ALL torch.nn classes (Conv, MaxPool2d, Upsample, etc.)
add_all_classes_from_module(nn)
add_all_classes_from_module(nn.modules)
add_all_classes_from_module(nn.modules.pooling)
add_all_classes_from_module(nn.modules.upsampling)
add_all_classes_from_module(nn.modules.activation)

print("✅ All YOLOv5 + torch.nn classes registered safely.")

# ==============================
# 4️⃣ GPU Check
# ==============================
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# ==============================
# 5️⃣ Upload image
# ==============================
uploaded = files.upload()
image_path = list(uploaded.keys())[0]

# ==============================
# 6️⃣ Load YOLOv5 license plate model
# ==============================
model = yolov5.load('keremberke/yolov5m-license-plate')

# ==============================
# 7️⃣ Run detection
# ==============================
results = model(image_path)
results.print()
results.show()

# ==============================
# 8️⃣ Crop detected plates
# ==============================
img = cv2.imread(image_path)
plate_images = []
for *box, conf, cls in results.xyxy[0].cpu().numpy():
    x1, y1, x2, y2 = map(int, box)
    plate_images.append(img[y1:y2, x1:x2])

# ==============================
# 9️⃣ OCR
# ==============================
reader = easyocr.Reader(['en'], gpu=(device=='cuda'))
for i, plate in enumerate(plate_images):
    print(f"\nPlate {i+1}:")
    text = " | ".join([d[1] for d in reader.readtext(plate)])
    print("Detected Text:", text)
    plt.imshow(cv2.cvtColor(plate, cv2.COLOR_BGR2RGB))
    plt.axis(False)
    plt.show()





✅ All YOLOv5 + torch.nn classes registered safely.
Using device: cuda


Saving plate.jpg to plate (11).jpg


/usr/local/lib/python3.11/dist-packages/yolov5/models/common.py:709: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
image 1/1: 742x1200 1 license_plate
Speed: 13.1ms pre-process, 20.5ms inference, 1.6ms NMS per image at shape (1, 3, 416, 640)


Showing images is not supported in this environment: OpenCV(4.12.0) :-1: error: (-5:Bad argument) in function 'rectangle'
> Overload resolution failed:
>  - img marked as output argument, but provided NumPy array marked as readonly
>  - argument for rectangle() given by name ('thickness') and position (4)
>  - Expected Ptr<cv::UMat> for argument 'img'
>  - argument for rectangle() given by name ('thickness') and position (4)


Plate 1:
Detected Text: SN66 XMZ
